In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm

from astropy.io import fits

## Creating Labels

### (1) Detection matches catalog entry

### Catalog

In [7]:
CAT_PATH = 'data/stationary_catalogs/stationary_catalog_0931_g.fits' # or whatever path
cat = fits.open(CAT_PATH)
# later incorporate these headers somehow?
# cat[0].header 

Extract catalog data into numpy ndarray (This is pretty quick)

In [109]:
def cat_to_ndarray(cat):
    ''' Input: cat - opened .fits catalog file
        Output: X - NxM ndarray (N=num entries, M=relavant columns)
                cat_cols - column names'''
    cat_cols = cat[1].data.columns.names
    cat_cols = cat_cols[:3] # only interested in first 3 cols

    data = cat[1].data
    X = np.array([data[c] for c in cat_cols]).T # ndarray of the 3 cols
    return X, cat_cols

In [110]:
with fits.open(CAT_PATH) as cat:
    cat_X, cat_cols = cat_to_ndarray(cat)

cat_df = pd.DataFrame(cat_X, columns=cat_cols)
cat_df.head()

,RA,Dec,MAG
0,215.580332,-19.576746,17.720699
1,215.581094,-19.683190,22.907400
2,215.580419,-19.629544,23.144400
3,215.580366,-19.622345,23.122900
4,215.581000,-19.700400,20.313400


### Detections

In [9]:
chips=['XY01', 'XY02', 'XY03', 'XY04', 'XY05', 'XY06',
       'XY10', 'XY11', 'XY12', 'XY13', 'XY14', 'XY15', 'XY16', 'XY17',
       'XY20', 'XY21', 'XY22', 'XY23', 'XY24', 'XY25', 'XY26', 'XY27',
       'XY30', 'XY31', 'XY32', 'XY33', 'XY34', 'XY35', 'XY36', 'XY37',
       'XY40', 'XY41', 'XY42', 'XY43', 'XY44', 'XY45', 'XY46', 'XY47',
       'XY50', 'XY51', 'XY52', 'XY53', 'XY54', 'XY55', 'XY56', 'XY57',
       'XY60', 'XY61', 'XY62', 'XY63', 'XY64', 'XY65', 'XY66', 'XY67',
       'XY71', 'XY72', 'XY73', 'XY74', 'XY75', 'XY76']

In [10]:
# Don't need all the columns
# hdus[chips[0]+'.psf'].columns

Extract detection data into numpy ndarray (This is not so quick, but workable)

In [111]:
def smf_to_ndarray(hdus):
    columns = ['RA_PSF', 'DEC_PSF', 'CAL_PSF_MAG', 'CAL_PSF_MAG_SIG']
    chip_data = []
    for chip in chips:
        data = hdus[chip+'.psf'].data
        data = np.array([data[c] for c in columns]).T
        chip_data.append(data)
    X = np.concatenate(chip_data, axis=0)
    return X, columns

In [112]:
SMF_PATH = 'data/smf/o6771g0234o.729155.cm.943415.smf'

with fits.open(SMF_PATH) as hdus:
    detect_X, columns = smf_to_ndarray(hdus)
    
detect_df = pd.DataFrame(detect_X, columns=columns)
detect_df.head()

,RA_PSF,DEC_PSF,CAL_PSF_MAG,CAL_PSF_MAG_SIG
0,210.331740,-8.423737,13.158958,0.048666
1,210.338067,-8.420269,12.207523,0.048666
2,210.189832,-8.613739,13.500641,0.048666
3,210.447618,-8.505982,13.075476,0.048666
4,210.256990,-8.457488,13.271885,0.048666


### Clean Data

remove NaNs and Infinities

In [115]:
def clean(X):
    '''Remove all rows from ndarray if any entries are nan or inf'''
    isnan = np.isnan(X).any(axis=1)
    isinf = np.isinf(X).any(axis=1)
    bad_idxs = np.any([isnan, isinf], axis=0)
    cleaned = X[~bad_idxs]
    return cleaned

In [116]:
detect_X = clean(detect_X)

### Compare Detection to Catalog

Detections have a CAL_PSF_MAG_SIGMA that we might want to use (rather than arbitrary tolerance) when checking if a magnitutes are "matches" but this is more involved.

In [26]:
# a sketch...
# basically create a gaussian with magnitute mu and sigma from detection
# and see if a catalog entry is within a given percentile of that gaussian
mu = detect_X[:,2]
sig = detect_X[:,3]
PERCENTILE = 0.95
conf_ints = norm(mu,sig).interval(PERCENTILE)
conf_ints = np.array(conf_ints).T
cat_mag = cat_df['MAG']
for lower, upper in conf_ints:
    lower <= cat_mag[0] <= upper # not complete

Simpler:

Compare detection and catalog entry elements (RA, DEC, MAG) using allclose()
and arbitrary tolerance

But this process appears *far too slow!* :'(

In [153]:
TOLERANCE = 1e-05
def get_comp_func(d):
    ''' Creates a comparison function for a given detection
        It can then be applied across the catalog'''
    d = d[:3] #only concerned with RA, DEC, and MAG
    def comp_func(c):
        return np.allclose(d, c, rtol=TOLERANCE)
    return comp_func

In [154]:
def check_all(compare, cats):
    for c in cats:
        if compare(c):
            return 1
    return 0

In [159]:
results = []
for d in detect_X[:10]:
    compare = get_comp_func(d)
    result = check_all(compare, cat_X)
    results.append(result)

# z = np.apply_along_axis(compare, 1, cat_X[:50000])

In [156]:
results

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
np.savetxt('results.csv', results, delimiter=',')